# Setup

In [2]:
# Google Only
from google.colab import drive
drive.mount('/content/drive')
ROOT_FOLDER = '/content/drive/My Drive/Code/autocomplete_me/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
## Set Variables for Local and Cloud File Finding
import os
import sys
sys.path.append(ROOT_FOLDER)

In [4]:
!ls -l '/content/drive/My Drive/Code/autocomplete_me/src'

total 27
-rw------- 1 root root 2516 Jul 14 22:20 predict_utils.py
drwx------ 2 root root 4096 Jul 11 13:20 __pycache__
-rw------- 1 root root 3340 Jul 15 12:47 reader.py
-rw------- 1 root root 3580 Jul 14 22:20 train_model_baseline.py
-rw------- 1 root root 3203 Jul 14 22:20 train_utils.py
-rw------- 1 root root 9341 Jul 12 14:24 utils.py


In [5]:
from src import utils, reader, predict_utils, train_utils
from importlib import reload
reload(utils)
reload(reader)
reload(predict_utils)
reload(train_utils)

Using TensorFlow backend.


<module 'src.train_utils' from '/content/drive/My Drive/Code/autocomplete_me/src/train_utils.py'>

## Load Text Data

In [6]:
text = reader.read_bbc('business')
content_type = 'BBC-Business'

In [7]:
text[0]

'UK economy facing \'major risks\'\n\nThe UK manufacturing sector will continue to face "serious challenges" over the next two years, the British Chamber of Commerce (BCC) has said.\n\nThe group\'s quarterly survey of companies found exports had picked up in the last three months of 2004 to their best levels in eight years. The rise came despite exchange rates being cited as a major concern. However, the BCC found the whole UK economy still faced "major risks" and warned that growth is set to slow. It recently forecast economic growth will slow from more than 3% in 2004 to a little below 2.5% in both 2005 and 2006.\n\nManufacturers\' domestic sales growth fell back slightly in the quarter, the survey of 5,196 firms found. Employment in manufacturing also fell and job expectations were at their lowest level for a year.\n\n"Despite some positive news for the export sector, there are worrying signs for manufacturing," the BCC said. "These results reinforce our concern over the sector\'s p

# Modelling

## Process Text Data

In [10]:
sequences, num_words, word_idx, idx_word = train_utils.preprocess_text(text)

In [32]:
features, labels = train_utils.pass_sliding_window(sequences, sequence_len=10)

There are 165294 sequences.


In [33]:
labels = train_utils.one_hot_labels_and_improve_efficiency(labels)

Labels matrix shape:  (165294, 12675)
Labels matrix shape:  (165294, 12675)


In [34]:
# Create Test Train Set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42, shuffle=True)

In [35]:
import gc
gc.enable()
del labels
gc.collect()

0

In [36]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (132235, 10)
X_test shape:  (33059, 10)
y_train shape:  (132235, 12675)
y_test shape:  (33059, 12675)


In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                       y_train:  1.4 GiB
                        y_test: 355.5 MiB
                      features: 12.6 MiB
                       X_train: 10.1 MiB
                        X_test:  2.5 MiB
                      word_idx: 576.1 KiB
                      idx_word: 576.1 KiB
                     sequences:  4.5 KiB
                          text:  4.2 KiB
                            __:  778.0 B


In [37]:
# Embedding Matrix
# embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/Users/jaipancholi/data/glove.6B.100d.txt')
embedding_matrix = utils.create_embedding_matrix(word_idx, num_words, '/content/drive/My Drive/Code/autocomplete_me/data/glove.6B.100d.txt')
embedding_matrix

Glove Vectors loading with dimension 100
There were 1387 words without pre-trained embeddings.


/content/drive/My Drive/Code/autocomplete_me/src/utils.py:180: RuntimeWarning:

invalid value encountered in true_divide



array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00656124, -0.04206555,  0.12508174, ..., -0.02506376,
         0.14220549,  0.04648907],
       [-0.02940788,  0.00775488,  0.02958461, ..., -0.0617054 ,
         0.07386386, -0.02477734],
       ...,
       [ 0.03117583, -0.01426646, -0.09698666, ..., -0.0022697 ,
         0.1283632 ,  0.13427433],
       [ 0.09864704, -0.09904199, -0.07051982, ...,  0.01649705,
        -0.12355457,  0.12704042],
       [ 0.14297517,  0.01937677, -0.11857604, ..., -0.00369095,
        -0.25356799, -0.00142062]])

In [38]:
embedding_matrix.shape
# temp_embedding_matrix = embedding_matrix[:-1, :]
# temp_embedding_matrix.shape

(12675, 100)

# Design Model

In [22]:
from tensorflow.keras.models import load_model

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [24]:
def train_model(filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=False, epochs=100):
  if not model and not use_pretrained_model:
    print('Provide one of either model or use_pretrained_model.')
  elif model and use_pretrained_model:
      print('Provide one of either model or use_pretrained_model.')
  elif use_pretrained_model:
    model = load_model(model_filepath)
  
  callbacks = [
      EarlyStopping(monitor='val_accuracy', patience=25),
      ModelCheckpoint(f'{model_filepath}', save_best_only=True, save_weights_only=False, monitor='val_accuracy')
  ]

  history = model.fit(
      X_train, 
      y_train, 
      epochs=epochs, 
      batch_size=2048, 
      validation_data=(X_test, y_test), 
      verbose=1,
      callbacks=callbacks
  )

  return history

##V1

In [28]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    # weights=[temp_embedding_matrix],
    trainable=True)
)

model.add(LSTM(64))

model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-1.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

Epoch 1/500


ValueError: ignored

## V2

In [39]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256))

model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-2.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

Epoch 1/500
65/65 [==============================] - 9s 132ms/step - loss: 7.6509 - accuracy: 0.0622 - val_loss: 7.1367 - val_accuracy: 0.0641
Epoch 2/500
65/65 [==============================] - 8s 117ms/step - loss: 7.0544 - accuracy: 0.0646 - val_loss: 7.0874 - val_accuracy: 0.0641
Epoch 3/500
65/65 [==============================] - 8s 117ms/step - loss: 6.9668 - accuracy: 0.0646 - val_loss: 7.0468 - val_accuracy: 0.0641
Epoch 4/500
65/65 [==============================] - 8s 127ms/step - loss: 6.8907 - accuracy: 0.0688 - val_loss: 6.9774 - val_accuracy: 0.0737
Epoch 5/500
65/65 [==============================] - 8s 121ms/step - loss: 6.7650 - accuracy: 0.0744 - val_loss: 6.8847 - val_accuracy: 0.0822
Epoch 6/500
65/65 [==============================] - 8s 121ms/step - loss: 6.6304 - accuracy: 0.0836 - val_loss: 6.8216 - val_accuracy: 0.0869
Epoch 7/500
65/65 [==============================] - 8s 120ms/step - loss: 6.5102 - accuracy: 0.0910 - val_loss: 6.7701 - val_accuracy: 0.0920

## V3

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-3.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

Epoch 1/500
49/49 [==============================] - 6s 126ms/step - loss: 7.8511 - accuracy: 0.0482 - val_loss: 7.2717 - val_accuracy: 0.0643
Epoch 2/500
49/49 [==============================] - 6s 116ms/step - loss: 7.1611 - accuracy: 0.0653 - val_loss: 7.2390 - val_accuracy: 0.0643
Epoch 3/500
49/49 [==============================] - 6s 118ms/step - loss: 7.0862 - accuracy: 0.0653 - val_loss: 7.1782 - val_accuracy: 0.0643
Epoch 4/500
49/49 [==============================] - 6s 129ms/step - loss: 7.0221 - accuracy: 0.0653 - val_loss: 7.1771 - val_accuracy: 0.0643
Epoch 5/500
49/49 [==============================] - 6s 118ms/step - loss: 6.9842 - accuracy: 0.0653 - val_loss: 7.1751 - val_accuracy: 0.0643
Epoch 6/500
49/49 [==============================] - 6s 129ms/step - loss: 6.9300 - accuracy: 0.0673 - val_loss: 7.1258 - val_accuracy: 0.0729
Epoch 7/500
49/49 [==============================] - 6s 119ms/step - loss: 6.8353 - accuracy: 0.0724 - val_loss: 7.0439 - val_accuracy: 0.0773

## V4

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.2))

model.add(LSTM(64))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-4.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

## V5

In [41]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.2))

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-5.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

Epoch 1/500
65/65 [==============================] - 16s 250ms/step - loss: 7.5981 - accuracy: 0.0612 - val_loss: 7.1419 - val_accuracy: 0.0641
Epoch 2/500
65/65 [==============================] - 15s 234ms/step - loss: 7.0865 - accuracy: 0.0646 - val_loss: 7.1518 - val_accuracy: 0.0641
Epoch 3/500
65/65 [==============================] - 16s 242ms/step - loss: 7.0693 - accuracy: 0.0646 - val_loss: 7.1983 - val_accuracy: 0.0641
Epoch 4/500
65/65 [==============================] - 15s 232ms/step - loss: 7.0584 - accuracy: 0.0646 - val_loss: 7.1747 - val_accuracy: 0.0641
Epoch 5/500
65/65 [==============================] - 15s 231ms/step - loss: 6.9775 - accuracy: 0.0647 - val_loss: 7.0458 - val_accuracy: 0.0641
Epoch 6/500
65/65 [==============================] - 16s 243ms/step - loss: 6.8599 - accuracy: 0.0705 - val_loss: 6.9444 - val_accuracy: 0.0735
Epoch 7/500
65/65 [==============================] - 16s 244ms/step - loss: 6.7380 - accuracy: 0.0758 - val_loss: 6.8965 - val_accuracy:

## V6

In [ ]:
model = Sequential()

model.add(    
    Embedding(
    input_dim=num_words,
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    trainable=True)
)

model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

model.add(Dropout(0.5))

model.add(LSTM(256))

model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))

# output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-6.h5')
train_model(model_filepath, X_train, y_train, X_test, y_test, use_pretrained_model=False, model=model, epochs=500)

In [ ]:
train_utils.plot_history(history)

# Comparison

### 100 Epochs

<tr>
    <th>LSTM Layers</th>
    <th>LSTM Cells per Layer</th>
    <th>Dropout %</th>
    <th>Validation Loss</th>
    <th>Validation Accuracy</th>
</tr>
<tr>
    <td>1</td>
    <td>64</td>
    <td>0.2</td>
    <td>8.9918</td>
    <td>0.2271</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.2</td>
    <td>10.7950</td>
    <td>0.2354</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.5</td>
    <td>8.9682</td>
    <td>0.2153</td>
</tr>
<tr>
    <td>2</td>
    <td>64</td>
    <td>0.2, 0.5</td>
    <td>6.9549</td>
    <td>0.1490</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.2, 0.5</td>
    <td>7.4581</td>
    <td>0.1683</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.5, 0.5</td>
    <td>7.3286</td>
    <td>0.1650</td>
</tr>

### 500 Epochs

<tr>
    <th>LSTM Layers</th>
    <th>LSTM Cells per Layer</th>
    <th>Dropout %</th>
    <th>Validation Loss</th>
    <th>Validation Accuracy</th>
</tr>
<tr>
    <td>1</td>
    <td>64</td>
    <td>0.2</td>
    <td>16.4351</td>
    <td>0.3229</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.2</td>
    <td>22.5922</td>
    <td>0.3373</td>
</tr>
<tr>
    <td>1</td>
    <td>256</td>
    <td>0.5</td>
    <td>18.8511</td>
    <td>0.3329</td>
</tr>
<tr>
    <td>2</td>
    <td>64</td>
    <td>0.2, 0.5</td>
    <td>8.8127</td>
    <td>0.2230</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.2, 0.5</td>
    <td>10.8240</td>
    <td>0.2854</td>
</tr>
<tr>
    <td>2</td>
    <td>256</td>
    <td>0.5, 0.5</td>
    <td>10.4388</td>
    <td>0.2666</td>
</tr>

# Generate Text Data

### Load Objects To Infer

In [11]:
from tensorflow.keras.models import load_model
model_filepath = os.path.join(ROOT_FOLDER, 'models', f'{content_type}-custom-2.h5')
model = load_model(model_filepath)
TRAINING_LENGTH = 10

## Existing Sentences

In [20]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=20,
    diversity=1,
    n_gen=1
)

/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py:42: RuntimeWarning:

divide by zero encountered in log



In [21]:
' '.join(word for word in original_sequence)

'retirement under 65 employers will no longer be able to'

In [22]:
' '.join(word for word in gen_list[0])

"< --- > advise for the companies said mr rubinsohn is due to delivering improvements and the us's biggest manufacturers could be above"

In [23]:
' '.join(word for word in a)

'< --- > force workers to retire before 65 unless they can justify it the government has announced that firms will be barred'

In [40]:
original_sequence, gen_list, a = predict_utils.generate_output(
    model,
    sequences,
    idx_word,
    seed_length=TRAINING_LENGTH,
    new_words=20,
    diversity=1,
    n_gen=1
)

/content/drive/My Drive/Code/autocomplete_me/src/predict_utils.py:42: RuntimeWarning:

divide by zero encountered in log



In [41]:
' '.join(word for word in original_sequence)

'surging and there are many companies who will continue with'

In [42]:
' '.join(word for word in gen_list[0])

'< --- > forecasts by 90 of its known manufacturing making currently frozen goods the firm recently indicated that the merger for an'

In [43]:
' '.join(word for word in a)

'< --- > existing trading relationships christian aid has called on british firms not to simply cut and run but look after their'

## Custom Sentences

In [ ]:
sentence = 'Stocks of major large technology firms are becoming even more fragile even though'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=20)

[None, 3, 546, 490, 45, 126, 13, 518, 150, 24, 9544, 150, 456]


ValueError: ignored

In [ ]:
sentence = 'However, there have been many instances of'
predict_utils.generate_custom_sentence(sentence, word_idx, idx_word, model, new_words=50)

[None, 56, 18, 46, 67, 7424, 3]


ValueError: ignored